In [1]:
from collections import defaultdict
import glob
import json
import os
import statistics

In [2]:
SPENT_OUTPUTS=True

## Load data, prep for analysis

In [3]:
# Load from included data files
blocks = {}
for file_path in glob.glob('./data/blocks-*.json'):
    with open(file_path, 'r') as file:
        data = json.load(file)['blocks']
        blocks.update(data)

with open('./data/spent-outputs.json', 'r') as f:
    spent_outputs = json.load(f)['outputs']

In [4]:
print(len(blocks))
print(len(spent_outputs))

188885
446235


In [5]:
chainblocks = set()
non_chainblocks = set()
merged_blues = set()
merged_reds = set()

blocks_per_daa = defaultdict(int)
block_timestamps = []
chainblock_timestamps = []

coinbase_txs = 0
coinbase_outputs = []

accepted_txs = set() # regular, non-coinbase
outputs_spent = [] # regular, non-coinbase. not that coinbase can spend utxos anyways! 
outputs_created = [] # regular, non-coinbase
fees = []

sending_addrs = set()
receiving_addrs = set()

In [6]:
# Pre-process data
for hash, block in blocks.items():
    blocks_per_daa[block['header']['daaScore']] += 1
    block_timestamps.append(int(block['header']['timestamp']))

    # Chainblock vs. non-chainblock
    if block['verboseData'].get('isChainBlock'):
        chainblocks.add(hash)
        chainblock_timestamps.append(int(block['header']['timestamp']))
    else:
        non_chainblocks.add(hash)

    # Blue mergeset 
    for bh in block['verboseData'].get('mergeSetBluesHashes', []):
        merged_blues.add(bh)

    # Red mergeset
    for bh in block['verboseData'].get('mergeSetRedsHashes', []):
        merged_reds.add(bh)

    # Process transactions
    for tx in block['transactions']:
        tx_id = tx['verboseData']['transactionId']

        # Skip if tx is not accepted
        if not tx.get('accepted'):
            continue

        # Skip if tx has already been processed
        if tx_id in accepted_txs:
            continue
        accepted_txs.add(tx_id)

        # Process coinbase transactions
        if tx['subnetworkId'] == '0100000000000000000000000000000000000000':
            coinbase_txs += 1
            for output in tx['outputs']:
                coinbase_outputs.append(int(output['amount']))
            continue

        # Process "regular" transactions
        total_output_amount = 0
        for output in tx['outputs']:
            outputs_created.append(int(output['amount']))
            total_output_amount += int(output['amount'])

            receiving_addrs.add(output['verboseData']['scriptPublicKeyAddress'])
            
        if SPENT_OUTPUTS:
            total_input_amount = 0
            for input in tx['inputs']:
                prev_outpoint_tx_id = input['previousOutpoint']['transactionId']
                prev_outpoint_index = input['previousOutpoint'].get('index', 0)
    
                input_amount = spent_outputs[f"{prev_outpoint_tx_id}-{prev_outpoint_index}"]['amount']        
                outputs_spent.append(input_amount)
                total_input_amount += input_amount
    
                sending_addr = spent_outputs[f"{prev_outpoint_tx_id}-{prev_outpoint_index}"]['address']
                sending_addrs.add(sending_addr)

            tx_fee = total_input_amount - total_output_amount
            fees.append(tx_fee)

## Results

In [7]:
def list_stats(title, l, ptotal=True, to_kas=True):
    l.sort()
    
    total = sum(l)
    
    mean = sum(l) / len(l)

    median = statistics.median(l)
    
    minimum = min(l)
    maximum = max(l)

    f = 100_000_000 if to_kas else 1
    
    print(title)
    if ptotal:
        print(f'Total: {total / f:,}')
    print(f'Mean: {mean / f:,}')
    print(f'Median: {median / f:,}')
    print(f'Min: {minimum / f:,}')
    print(f'Max: {maximum / f:,}')

In [8]:
print('--- COUNTS')
print(f'Chainblocks: {len(chainblocks):,}')
print(f'Non-chainblocks: {len(non_chainblocks):,}')
print(f'Merged blues: {len(merged_blues):,}')
print(f'Merged reds: {len(merged_reds):,}\n')
print(f'Coinbase transactions: {coinbase_txs:,}')
print(f'Coinbase outputs: {len(coinbase_outputs):,}\n')
print(f'Accepted transactions (not incl. coinbase): {len(accepted_txs):,}')
print(f'Outputs spent: {len(outputs_spent):,}')
print(f'Outputs created: {len(outputs_created):,}\n')
print(f'Fees: {len(fees):,} (qty of fees should = accepted txs - coinbase txs)\n')
print(f'Unique sending addresses: {len(sending_addrs):,}')
print(f'Unique receiving addresses: {len(receiving_addrs):,}')

--- COUNTS
Chainblocks: 100,409
Non-chainblocks: 88,476
Merged blues: 188,519
Merged reds: 415

Coinbase transactions: 100,407
Coinbase outputs: 188,796

Accepted transactions (not incl. coinbase): 226,725
Outputs spent: 446,230
Outputs created: 242,425

Fees: 126,318 (qty of fees should = accepted txs - coinbase txs)

Unique sending addresses: 32,268
Unique receiving addresses: 71,957


In [9]:
list_stats('--- Coinbase Outputs (in KAS)', coinbase_outputs)

--- Coinbase Outputs (in KAS)
Total: 27,734,234.75331338
Mean: 146.9005421370865
Median: 146.83238395
Min: 146.83238395
Max: 587.3296358


In [10]:
list_stats('--- Spent Outputs (in KAS)', outputs_spent)

--- Spent Outputs (in KAS)
Total: 3,196,990,675.2728896
Mean: 7,164.445858128968
Median: 146.83238395
Min: 6e-06
Max: 43,949,603.79212584


In [11]:
list_stats('--- Created Outputs (in KAS)', outputs_created)

--- Created Outputs (in KAS)
Total: 3,196,990,581.697504
Mean: 13,187.544938424273
Median: 87.25228395
Min: 7.3e-06
Max: 43,949,603.79212584


In [12]:
list_stats('--- Fees (in KAS)', fees)

--- Fees (in KAS)
Total: 93.57538553
Mean: 0.0007407921715828306
Median: 0.0003
Min: 1.635e-05
Max: 5.00005522


In [13]:
bpd = list(blocks_per_daa.values())
bpd.sort()
list_stats('--- Blocks per DAA', bpd, ptotal=False, to_kas=False)

--- Blocks per DAA
Mean: 1.6262581039544715
Median: 1.0
Min: 1.0
Max: 12.0


In [14]:
block_timestamps.sort()
block_timestamp_diffs = [block_timestamps[i+1] - block_timestamps[i] for i in range(len(block_timestamps) - 1)]
list_stats('--- Block intervals (in milliseconds)', block_timestamp_diffs, ptotal=False, to_kas=False)

--- Block intervals (in milliseconds)
Mean: 998.8845111285233
Median: 727.0
Min: 0.0
Max: 13,876.0


In [15]:
chainblock_timestamps.sort()
chainblock_timestamp_diffs = [chainblock_timestamps[i+1] - chainblock_timestamps[i] for i in range(len(chainblock_timestamps) - 1)]
list_stats('--- Chainblock intervals (in milliseconds)', chainblock_timestamp_diffs, ptotal=False, to_kas=False)

--- Chainblock intervals (in milliseconds)
Mean: 1,879.046181579157
Median: 1,534.0
Min: 8.0
Max: 18,527.0
